### [1203\. Sort Items by Groups Respecting Dependencies](https://leetcode.com/problems/sort-items-by-groups-respecting-dependencies/)

Difficulty: **Hard**


There are `n` items each belonging to zero or one of `m` groups where `group[i]` is the group that the `i`-th item belongs to and it's equal to `-1` if the `i`-th item belongs to no group. The items and the groups are zero indexed. A group can have no item belonging to it.

Return a sorted list of the items such that:

*   The items that belong to the same group are next to each other in the sorted list.
*   There are some relations between these items where `beforeItems[i]` is a list containing all the items that should come before the `i`-th item in the sorted array (to the left of the `i`-th item).

Return any solution if there is more than one solution and return an **empty list** if there is no solution.

**Example 1:**

**![](https://assets.leetcode.com/uploads/2019/09/11/1359_ex1.png)**

```
Input: n = 8, m = 2, group = [-1,-1,1,0,0,1,0,-1], beforeItems = [[],[6],[5],[6],[3,6],[],[],[]]
Output: [6,3,4,1,5,2,0,7]
```

**Example 2:**

```
Input: n = 8, m = 2, group = [-1,-1,1,0,0,1,0,-1], beforeItems = [[],[6],[5],[6],[3],[],[4],[]]
Output: []
Explanation: This is the same as example 1 except that 4 needs to be before 6 in the sorted list.
```

**Constraints:**

*   `1 <= m <= n <= 3*10^4`
*   `group.length == beforeItems.length == n`
*   `-1 <= group[i] <= m-1`
*   `0 <= beforeItems[i].length <= n-1`
*   `0 <= beforeItems[i][j] <= n-1`
*   `i != beforeItems[i][j]`
*   `beforeItems[i] `does not contain duplicates elements.

In [81]:
# Hierarchy Topological sorting @neal_wu
# aka. if there is a cycle in DAG
# Easier problem: 207. Course Scheduling
# 1) check DAG inside the group
# 2) check DAG between groups (each group or ungrouped individual is a node)
# Note: beforeItems -> the directed edges
# Runtime: 844 ms
# Memory Usage: 37.9 MB
# https://leetcode.com/problems/sort-items-by-groups-respecting-dependencies/discuss/387601/chinesec-1203-tuo-bu-pai-xu-on
# https://www.geeksforgeeks.org/topological-sorting-indegree-based-solution/
from typing import List, Dict
from collections import defaultdict

class Solution:
    def sortItems(self, n: int, m: int, group: List[int], beforeItems: List[List[int]]) -> List[int]:
        # Kahn's algorithm
        def tsort(vertices: List[int], edges: List[List[int]]) -> (List[int], bool):
            neighbors = defaultdict(list)
            for u, v in edges:
                neighbors[u].append(v)
            
            indegree = defaultdict(int)
            order = []
            # find "start nodes" which have no incoming edges
            for v in vertices:
                for u in neighbors[v]:
                    indegree[u] += 1
            for v in vertices:
                if indegree[v] == 0:
                    order.insert(0, v) # enqueue
            
            cur = 0
            while cur < len(order):
                v = order[cur]
                cur += 1
                for u in neighbors[v]:
                    indegree[u] -= 1
                    if indegree[u] == 0:
                        order.insert(0, u)
            return (order, cur == len(vertices))
        
        # trick
        for i in range(n):
            if group[i] < 0: # each ungrouped individual (-1) is actually a seperate group
                group[i] = m # regroup their group id from m, m+1, ...
                m += 1
        
        # group # -> vertices
        in_group_vertices = [[] for _ in range(m)]
        for i, g in enumerate(group):
            in_group_vertices[g].append(i)
        
        # classify edges to in-group-edge and cross-group-edge
        in_group_edges = [[] for _ in range(m)] # in-groups of edges
        group_edges = [] # across group edges
        for i, (g, b4) in enumerate(zip(group, beforeItems)):
            for b in b4:
                if group[b] == g: # in group edge
                    in_group_edges[g] += (i, b), # directed in-group edge: curNode i -> beforeNode b
                else: # cross group edge
                    group_edges += (g, group[b]), # directed cross-group edge: curGroup g -> beforeGroup group[b]
        
        # 1. in group tsort
        in_group_ordering = [[] for _ in range(m)]
        for g in range(m):
            in_group_ordering[g], is_acyclic = tsort(in_group_vertices[g], in_group_edges[g])
            if not is_acyclic: return []
        
        # 2. cross group tsort
        group_vertices = [i for i in range(m)]
        group_ordering, is_acyclic = tsort(group_vertices, group_edges)
        if not is_acyclic: return []
        
        # flatten list of list, equivalent to: 
        # for g in group_ordering:
        #     ans += in_group_ordering[g]
        ans = [x for g in group_ordering for x in in_group_ordering[g]]
        return ans

In [82]:
Solution().sortItems(n = 8, m = 2, group = [-1,-1,1,0,0,1,0,-1], beforeItems = [[],[6],[5],[6],[3,6],[],[],[]])

[6, 3, 4, 7, 1, 0, 5, 2]

In [83]:
Solution().sortItems(n = 8, m = 2, group = [-1,-1,1,0,0,1,0,-1], beforeItems = [[],[6],[5],[6],[3],[],[4],[]])

[]